In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import pandas as pd
import shutil
import jupyter_notebooks.utils.aux_functions as aux
from scipy.special import expit
from tqdm import tqdm

from typing import List, Tuple

In [6]:
def sigmoid_correction(image, threshold, factor=10):
    normalized_image = image / 255.0

    corrected_image = expit(factor * (normalized_image - threshold))
    
    corrected_image = (corrected_image * 255).astype(np.uint8)
    
    return corrected_image

def multi_sigmoid_correction(image, threshold_luz: float, threshold_sombra: float, factor_luz: int, factor_sombra: int, div: List[Tuple[int, int]]):
    normalized_image = image / 255.0
    corrected_image = np.zeros_like(normalized_image)
    
    for fila in range(int(image.shape[0])):
        for columna in range(int(image.shape[1])):
            
            if columna >= div[fila][0]:
                corrected_image[fila, columna, :] = expit(factor_luz * (normalized_image[fila, columna, :] - threshold_luz))
                
            else:
                corrected_image[fila, columna, :] = expit(factor_sombra * (normalized_image[fila, columna, :] - threshold_sombra))
    
    corrected_image = (corrected_image * 255).astype(np.uint8)
    
    return corrected_image
    
def get_m_n(points: List[Tuple[int, int]]):
    x1, y1, x2, y2 = points[0][0], points[0][1], points[1][0], points[1][1]
    
    if len(points) != 2:
        raise ValueError("La lista no contiene 2 puntos")
    
    else:
        m = (y2 - y1)/(x2 - x1)
        n = y1 - m * x1
    
    return m, n

In [3]:
# Para imagenes oscuras del estilo de 20240615001109 threshol = 0.7 y factor 19
# Para imagenes del estilo 20240615015622 threshol = 0.5 y factor 19
# Para imagenes del estilo 20240614235237 threshol = 0.5 y factor 19

# TIENE PINTA DE QUE AQUÍ MODIFICANDO THRESHOLS MANTENIENDO EL FACTOR A 19 VA A SER FACIL DE HACER UNA POR UNA
path_dest = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\aplay_sigmoid'
path_orig = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche'
path_agua_mask = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mascara_agua_lescala_2.jpg'
path_no_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\no_procesadas'

transparencia = 0.4
kernel_dil = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

factor = 19
threshold = 0.7


for name_im in os.listdir(path_orig):
    im_path = os.path.join(path_orig, name_im)
    #im_path_orig = os.path.join(path_orig, name_im)
    print(im_path)
    mask_ag = cv2.imread(path_agua_mask)
    _, mask_agua_bin = cv2.threshold(mask_ag, 128, 1, cv2.THRESH_BINARY)
    mask_agua = np.uint8(mask_agua_bin)
    mask_agua = cv2.resize(mask_agua, (512, 288))
        
    img = cv2.imread(im_path)
    img = cv2.resize(img, (512, 288))
        
    res = np.zeros_like(img)
    
    key = '  '
    while (key != ord('q') and key != ord('g') and key != ord('n')):
        
        res[:,:,0] = sigmoid_correction(img[:,:,0],threshold,factor)
        res[:,:,1] = sigmoid_correction(img[:,:,1],threshold,factor)
        res[:,:,2] = sigmoid_correction(img[:,:,2],threshold,factor)
        
        res = res*mask_agua
        
        imagenes = cv2.vconcat([res[:,:,0], res[:,:,1], res[:,:,2]])
        cv2.namedWindow(f'threshold: {threshold} Factor: {factor}', cv2.WINDOW_NORMAL)
        cv2.moveWindow(f'threshold: {threshold} Factor: {factor}', 3000, 0)
        cv2.resizeWindow(f'threshold: {threshold} Factor: {factor}', int(res.shape[1]), int((res.shape[0])*3))
        cv2.imshow(f'threshold: {threshold} Factor: {factor}', imagenes)
        
        cv2.namedWindow('Orig', cv2.WINDOW_NORMAL)
        cv2.moveWindow('Orig', 2000, 0)
        cv2.resizeWindow('Orig', int(res.shape[1]), int((res.shape[0])))
        cv2.imshow(f'Orig', img)
        
        key = cv2.waitKey(0)
        
        if key == ord('a'):
            threshold -= 0.05
            cv2.destroyAllWindows()
        
        if key == ord('s'):
            threshold += 0.05
            cv2.destroyAllWindows()
            
        if key == ord('d'):
            factor -= 1
            cv2.destroyAllWindows()
        
        if key == ord('f'):
            factor += 1
            cv2.destroyAllWindows()
        
        if key == ord('g'):
            cv2.imwrite(path_dest + '/'+ name_im, res)
            cv2.destroyAllWindows()
            
        if key == ord('n'):
            shutil.move(im_path, path_no_procesadas)
            cv2.destroyAllWindows()
        
        if key == ord('q'):
            cv2.destroyAllWindows()


C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234105.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234215.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234324.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234434.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234543.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234652.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234801.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614234910.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614235018.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614235127.tif
C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\20240614235237.tif
C:\Users\aquacorp\Des

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [7]:
path_img_sigTransform = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\aplay_sigmoid'
path_orig = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche'
dir_destino = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\mascaras'
almacen_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\noche\procesadas'

transparencia = 0.4
kernel_dil = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

thresh = 80#100

for name_im in os.listdir(path_img_sigTransform):
    im_path = os.path.join(path_img_sigTransform, name_im)
    im_path_orig = os.path.join(path_orig, name_im)
    
    img = cv2.imread(im_path) # Shape: (2160, 3840, 3)
    img = cv2.resize(img, (512, 288)) # Shape: (512, 288, 3)
    img_orig = cv2.imread(im_path_orig)
    img_orig = cv2.resize(img_orig, (512, 288))
    
    key = '  '
    while (key != ord('q') and key != ord('g')):
        mask = np.ones_like(img[:,:,1], dtype=np.uint8)
        mask_out_of_range = (img[:,:,1] < thresh)
        mask[mask_out_of_range] = 0
        
        mask = cv2.dilate(mask, kernel_dil, iterations=1)
        
        #mask_rgb = cv2.cvtColor(mask*255, cv2.COLOR_GRAY2RGB)   Esta linea es para tener la máscara en color blanco
        mask_rgb = np.zeros_like(img) # Estas lineas son para tener la máscara en color rojo
        mask_rgb[:, :, 2] = mask * 255  # Estas lineas son para tener la máscara en color rojo
        
        composicion_pred = cv2.addWeighted(mask_rgb, transparencia, img_orig, 1 - transparencia, 0)
        imagenes = cv2.vconcat([img_orig, mask_rgb, composicion_pred])
        
        cv2.namedWindow(str(name_im[:-4]) + ' ' + str(thresh), cv2.WINDOW_NORMAL)
        cv2.moveWindow(str(name_im[:-4]) + ' ' + str(thresh), 3000, 0)
        cv2.resizeWindow(str(name_im[:-4]) + ' ' + str(thresh), int(img.shape[1]), int((img.shape[0])*3))
        cv2.imshow(str(name_im[:-4]) + ' ' + str(thresh), imagenes)
        
        key = cv2.waitKey(0)
    
        if key == ord('a'):
            thresh = thresh - 3
            cv2.destroyAllWindows()
        
        if key == ord('s'): # subir limite inferior
            thresh = thresh + 3
            cv2.destroyAllWindows()
        
        if key == ord('g'): # guardar máscara
            dir_save_mask = dir_destino + '/m_' + name_im
            cv2.imwrite(dir_save_mask, mask)
            shutil.move(im_path, almacen_procesadas)
            cv2.destroyAllWindows()
            
        if key == ord('q'):  # sube limite inferior
            cv2.destroyAllWindows()